# Review Instructions

Please review the MSv4 `antenna_xds` schema and the XRADIO interface (`ps['MSv4_name'].antenna_xds`). Note that the PS (processing set) interface or the main_xds should not be reviewed.

The `antenna_xds` schema specification: https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=257301047

The processing set is a loose collection of MSv4 which might come from multiple MSv2 (or ASDMS). Consequently, arbitrary ids are avoided in favor of descriptive strings.

Two example datasets will be used VLBA_TL016B_split_lsrk.ms (VLBI) and uid___A002_X1015532_X1926f.small.ms (Single Dish).

## Preparatory Material
Go over Xarray nomenclature and selection syntax:
- https://docs.xarray.dev/en/latest/user-guide/terminology.html
- https://docs.xarray.dev/en/latest/user-guide/indexing.html

MSv2 and CASA documentation:
- MSv2 schema: https://casacore.github.io/casacore-notes/229.pdf
- MSv3 schema: https://casacore.github.io/casacore-notes/264.pdf

VLBI MSv2 extension: 
https://casacore.github.io/casacore-notes/265.pdf

## `antenna_xds` Schema
The ANTENNA, FEED, GAIN_CURVE, PHASE_CAL, and INTERFEROMETER_MODEL (VLBI, not yet included) tables in the MSv2 contain closely related information that is all related to the antenna. The MSv4 contains a single SPW, consequently, an antenna will only have a single feed associated with it.

Antenna_ids are no longer used in favor of the antenna name (for antennas that can move the name consists of the name + "_" + station). This allows for easily comparing MSv4s from different observations.

## Key Questions to Answer
### Schema Questions
- 1.1) Are there missing use cases?
- 1.2) Is all the information present needed for offline processing?
- 1.3) (VLBI) Instead of storing BASELINE_REFERENCE in main_xds can we store it in the antenna_xds? This would assume that for the duration of the MS v4 that the reference antennas remain constant.
- 1.4) Is the order of the dims correct (antenna_id)?
- 1.5) Should BEAM_OFFSET be sky_dir_label (Ra, Dec) or local_sky_label (Az, Alt)?
- 1.6) Should we add prefex to organize data variables? For example PHASE_DELAY -> VLBI_PHASE_DELAY?
- 1.7) Should we include the POLARIZATION_RESPONSE doesn't seem to be used?
- 1.8) Can we get rid of POLARIZATION_RESPONSE?
- 1.9) ANTENNA_FEED_OFFSET is a new data variable that is calculated using (Antenna_Table.offset + Feed_Table.position). Is this any need to store these values separately?

### XRADIO
2.1) Please check all the data variable (names,dimensions,measures) and coordinates (names,dimensions,measures) in both the google spreadsheet and ipynb.

# Environment instructions

It is recommended to use the conda environment manager to create a clean, self-contained runtime where xradio and all its dependencies can be installed:

```bash
conda create --name xradio python=3.11 --no-default-packages
conda activate xradio
```

Clone the repository, checkout the review branch and do a local install:

```bash
git clone https://github.com/casangi/xradio.git
git checkout 168-review-ms_xdsattrsantenna_xds-schema-and-xradio-interface
cd xradio
pip install -e .
```

On macOS it is required to pre-install python-casacore using ```bash conda install -c conda-forge python-casacore```.

# Data
Two examples will be given VLBA_TL016B_split_lsrk.ms (VLBI) and  uid___A002_X1015532_X1926f.small.ms (Single Dish). But you are welcome to change things up by downloading (or using your own MSv2 dataset). The GraphVIPER function list_files will give you a list of available datasets.


In [1]:
from graphviper.utils.data import list_files
list_files()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ file                                     ┃ dtype          ┃ telescope   ┃ size     ┃ mode                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ea25_cal_small_before_fixed.split.ms     │ CASA MS v2     │ VLA         │ 0.209    │ Holography                 │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ ea25_cal_small_after_fixed.split.ms      │ CASA MS v2     │ VLA         │ 0.209    │ Holography                 │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ J1924-2914.ms.calibrated.split.SPW3      │ CASA MS v2     │ ALMA        │ 0.055    │ Holography                 │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ extract_holog_verification.json          │ json           │ na          │ 0.000004 │ na                         │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ holog_numerical_verification.json        │ json           │ na          │ 0.000004 │ na                         │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ locit-input-pha.cal                      │ CASA Cal Table │ VLA         │ 0.000468 │ antenna location cal table │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ panel_cutoff_mask                        │ npy            │ na          │ 0.000348 │ na                         │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ heuristic_model                          │ bin            │ na          │ 0.000004 │ na                         │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ panel_test_files                         │ na             │ na          │          │ na                         │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ AA2-Mid-sim_00000.ms                     │ CASA MS v2     │ SKA Mid     │ 0.029    │ Simulated Interferometer   │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ Antennae_M8.img.zarr                     │ zarr           │ ALMA        │ 0.014    │ Interferometer Mosaic      │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ Antennae_North.cal.lsrk.ms               │ CASA MS v2     │ ALMA        │ 0.202    │ Interferometer Mosaic      │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ Antennae_North.cal.lsrk.py39.vis.zarr    │ zarr           │ ALMA        │ 0.207    │ Interferometer Mosaic      │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ Antennae_North.cal.lsrk.split.ms         │ CASA MS v2     │ ALMA        │ 0.0056   │ Interferometer Mosaic      │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ Antennae_North.cal.lsrk.split.py39.vis.… │ zarr           │ ALMA        │ 0.0047   │ Interferometer Mosaic      │
├──────────────────────────────────────────┼────────────────┼─────────────┼──────────┼────────────────────────────┤
│ complex_valued_uv.im                     │ image          │ Simulated   │ 0.768    │ Simulated                  │
├──────────────────────────────────────────┼────────────

# VLBI Example

## Download Dataset

In [2]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="VLBA_TL016B_split_lsrk.ms")

[2024-08-23 14:48:50,752]  WARNING  graphviper:  File exists: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/graphviper/utils/data/.dropbox 
[2024-08-23 14:48:50,752]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List              
 ─────────────────────────── 
  VLBA_TL016B_split_lsrk.ms

[2024-08-23 14:48:51,589]     INFO  graphviper:  File exists: VLBA_TL016B_split_lsrk.ms 


## Start Dask cluster 
Choose an approriate number of cores and memory_limit (this is per core).

In [3]:
from graphviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

[2024-08-23 14:48:51,651]     INFO  graphviper:  Checking parameter values for client.local_client 
[2024-08-23 14:48:51,651]     INFO  graphviper:  Module path: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11//site-packages/ 
[2024-08-23 14:48:51,654]  WARNING      client:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 
[2024-08-23 14:48:52,411]     INFO      client:  Created client <MenrvaClient: 'tcp://127.0.0.1:51104' processes=4 threads=4, memory=14.90 GiB> 


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51104,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:51117,Total threads: 1
Dashboard: http://127.0.0.1:51119/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:51107,


## Convert dataset

In [12]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

in_file = "VLBA_TL016B_split_lsrk.ms"
out_file = "VLBA_TL016B_split_lsrk.vis.zarr"

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    parallel=False,
    overwrite=True,
    phase_cal_interpolate=True,
)

[2024-08-23 14:49:22,915]     INFO      client:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-23 14:49:22,933]     INFO      client:  Number of partitions: 4 


[2024-08-23 14:49:22,934]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [-1], FIELD [0], SCAN [0] 
[2024-08-23 14:49:23,494]  WARNING      client:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-23 14:49:23,577]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [-1], FIELD [1], SCAN [0] 
[2024-08-23 14:49:23,726]  WARNING      client:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-23 14:49:23,828]     INFO      client:  OBSERVATION_ID [0], DDI [1], STATE [-1], FIELD [0], SCAN [0] 
[2024-08-23 14:49:23,961]  WARNING      client:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-23 14:49:24,047]     INFO      client:  OBSERVATION_ID [0], DDI [1], STATE [-1], FIELD [1], SCAN [0] 
[2024-08-23 14:49:24,196]  WARNING      client:  Source_id is -1. No source information will be included in the field_and_source_xds. 


## Inspect Processing Set

In [13]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("VLBA_TL016B_split_lsrk.vis.zarr")
ps.summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
2,VLBA_TL016B_split_lsrk_0,[obs_0],"(200, 55, 6, 2)","[RR, LL]",[0],spw_0,[4C39.25_0],[Unknown],[],"[fk5, 9h27m03.01s, 39d02m20.85s]",5.004196e+09,5.006697e+09
3,VLBA_TL016B_split_lsrk_1,[obs_0],"(540, 55, 6, 2)","[RR, LL]",[0],spw_0,[J1154+6022_1],[Unknown],[],"[fk5, 11h54m04.54s, 60d22m20.82s]",5.004196e+09,5.006697e+09
1,VLBA_TL016B_split_lsrk_2,[obs_0],"(200, 55, 6, 2)","[RR, LL]",[0],spw_1,[4C39.25_0],[Unknown],[],"[fk5, 9h27m03.01s, 39d02m20.85s]",5.068199e+09,5.070699e+09
0,VLBA_TL016B_split_lsrk_3,[obs_0],"(540, 55, 6, 2)","[RR, LL]",[0],spw_1,[J1154+6022_1],[Unknown],[],"[fk5, 11h54m04.54s, 60d22m20.82s]",5.068199e+09,5.070699e+09


## Inspect antenna_xds:

In [14]:
ant_xds = ps['VLBA_TL016B_split_lsrk_0'].attrs['antenna_xds'].load()
ant_xds

<xarray.Dataset> Size: 228kB
Dimensions:                   (antenna_name: 10, cartesian_pos_label: 3,
                               receptor_label: 2, sky_dir_label: 2,
                               poly_term: 1, time: 200, tone_label: 2)
Coordinates:
  * antenna_name              (antenna_name) <U2 80B 'BR' 'FD' ... 'PT' 'SC'
  * cartesian_pos_label       (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    gain_curve_type           (antenna_name) <U9 360B 'POWER(ZA)' ... 'POWER(...
    mount                     (antenna_name) <U6 240B 'ALT-AZ' ... 'ALT-AZ'
    polarization_type         (antenna_name, receptor_label) <U1 80B 'R' ... 'L'
  * receptor_label            (receptor_label) <U25 200B 'pol_0' 'pol_1'
  * sky_dir_label             (sky_dir_label) <U3 24B 'ra' 'dec'
    station                   (antenna_name) <U2 80B 'BR' 'FD' ... 'PT' 'SC'
  * time                      (time) float64 2kB 1.645e+09 ... 1.645e+09
Dimensions without coordinates: poly_term, tone_label
Data variables:
    ANTENNA_DISH_DIAMETER     (antenna_name) float64 80B 25.0 25.0 ... 25.0 25.0
    ANTENNA_FEED_OFFSET       (antenna_name, cartesian_pos_label) float64 240B ...
    ANTENNA_POSITION          (antenna_name, cartesian_pos_label) float64 240B ...
    BEAM_OFFSET               (antenna_name, receptor_label, sky_dir_label) float64 320B ...
    GAIN_CURVE                (antenna_name, receptor_label, poly_term) float32 80B ...
    GAIN_CURVE_INTERVAL       (antenna_name) float64 80B 2.601e+04 ... 2.601e+04
    GAIN_CURVE_SENSITIVITY    (antenna_name, receptor_label) float32 80B 0.11...
    PHASE_CAL                 (antenna_name, time, receptor_label, tone_label) complex128 128kB ...
    PHASE_CAL_CABLE_CAL       (antenna_name, time) float64 16kB nan nan ... nan
    PHASE_CAL_INTERVAL        (antenna_name, time) float64 16kB nan nan ... nan
    PHASE_CAL_TONE_FREQUENCY  (antenna_name, time, receptor_label, tone_label) float64 64kB ...
    RECEPTOR_ANGLE            (antenna_name, receptor_label) float64 160B 0.0...
Attributes:
    overall_telescope_name:  VLBA
    relocatable_antennas:    False

GAIN_CURVE(name, gain_curve_time, receptor_name, poly_term)
GAIN_CURVE_INTERVAL (name, gain_curve_time)
GAIN_CURVE_SENSITIVITY (name, gain_curve_time, receptor_name)

Proposal:
GAIN_CURVE(name,receptor_name, poly_term)
GAIN_CURVE_INTERVAL (name,)
GAIN_CURVE_SENSITIVITY (name,receptor_name)
gain_curve_time (name) 


antenna_xds

antenna, feed, pointing, sys_cal, phase_cal, gain_curve, beam (require zernike, polynomial, airy disk, url)






In [7]:
ant_xds.GAIN_CURVE

<xarray.DataArray 'GAIN_CURVE' (antenna_name: 10, receptor_label: 2,
                                poly_term: 1)> Size: 80B
array([[[1.],
        [1.]],

       [[1.],
        [1.]],

       [[1.],
        [1.]],

       [[1.],
        [1.]],

       [[1.],
        [1.]],

       [[1.],
        [1.]],

       [[1.],
        [1.]],

       [[1.],
        [1.]],

       [[1.],
        [1.]],

       [[1.],
        [1.]]], dtype=float32)
Coordinates:
  * antenna_name       (antenna_name) <U2 80B 'BR' 'FD' 'HN' ... 'OV' 'PT' 'SC'
    gain_curve_type    (antenna_name) <U9 360B 'POWER(ZA)' ... 'POWER(ZA)'
    mount              (antenna_name) <U6 240B 'ALT-AZ' 'ALT-AZ' ... 'ALT-AZ'
    polarization_type  (antenna_name, receptor_label) <U1 80B 'R' 'L' ... 'L'
  * receptor_label     (receptor_label) <U25 200B 'pol_0' 'pol_1'
    station            (antenna_name) <U2 80B 'BR' 'FD' 'HN' ... 'OV' 'PT' 'SC'
Dimensions without coordinates: poly_term

# Single Dish Example

## Convert Dataset

In [8]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="uid___A002_X1015532_X1926f.small.ms")

[2024-08-23 14:48:54,464]  WARNING      client:  File exists: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/graphviper/utils/data/.dropbox 
[2024-08-23 14:48:54,464]     INFO      client:  Updating file metadata information ...  
 

 Download List                        
 ───────────────────────────────────── 
  uid___A002_X1015532_X1926f.small.ms

[2024-08-23 14:48:55,274]     INFO      client:  File exists: uid___A002_X1015532_X1926f.small.ms 


## Convert Dataset

In [9]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

in_file = "uid___A002_X1015532_X1926f.small.ms"
out_file = "uid___A002_X1015532_X1926f.small.vis.zarr"

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    parallel=True,
    overwrite=True,
)

[2024-08-23 14:48:55,293]     INFO      client:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-23 14:48:55,325]     INFO      client:  Number of partitions: 20 
[2024-08-23 14:48:55,326]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [10], FIELD [1], SCAN [2 4] 
[2024-08-23 14:48:55,327]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [11], FIELD [1], SCAN [2 4] 
[2024-08-23 14:48:55,328]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [12], FIELD [1], SCAN [2 4] 
[2024-08-23 14:48:55,329]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45], FIELD [1], SCAN [3 5] 
[2024-08-23 14:48:55,329]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [14 16 18 20 22 24 26 28 30 32 34 36 38 40 42 44], FIELD [1], SCAN [3 5] 
[2024-08-23 14:48:55,329]     INFO      client:  OBSERVATION_ID [0], DDI [1], STATE [10], FIELD [1], SCAN [2 4] 
[2024-08-

## Inspect Processing Set

In [10]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("uid___A002_X1015532_X1926f.small.vis.zarr")
ps.summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
14,uid___A002_X1015532_X1926f.small_00,"[CALIBRATE_ATMOSPHERE#OFF_SOURCE, CALIBRATE_WV...","(30, 4, 4, 2)","[XX, YY]","[2, 4]",X780709176#ALMA_RB_07#BB_1#SW-01#FULL_RES_0,[Jupiter_1],[Jupiter_1],[SPW1_-_CO_v=0_3-2(ID=3768104)],Ephemeris,3.457649e+11,3.457650e+11
11,uid___A002_X1015532_X1926f.small_01,"[CALIBRATE_ATMOSPHERE#AMBIENT, CALIBRATE_WVR#A...","(30, 4, 4, 2)","[XX, YY]","[2, 4]",X780709176#ALMA_RB_07#BB_1#SW-01#FULL_RES_0,[Jupiter_1],[Jupiter_1],[SPW1_-_CO_v=0_3-2(ID=3768104)],Ephemeris,3.457649e+11,3.457650e+11
0,uid___A002_X1015532_X1926f.small_02,"[CALIBRATE_ATMOSPHERE#HOT, CALIBRATE_WVR#HOT]","(30, 4, 4, 2)","[XX, YY]","[2, 4]",X780709176#ALMA_RB_07#BB_1#SW-01#FULL_RES_0,[Jupiter_1],[Jupiter_1],[SPW1_-_CO_v=0_3-2(ID=3768104)],Ephemeris,3.457649e+11,3.457650e+11
3,uid___A002_X1015532_X1926f.small_03,[OBSERVE_TARGET#OFF_SOURCE],"(510, 4, 4, 2)","[XX, YY]","[3, 5]",X780709176#ALMA_RB_07#BB_1#SW-01#FULL_RES_0,[Jupiter_1],[Jupiter_1],[SPW1_-_CO_v=0_3-2(ID=3768104)],Ephemeris,3.457649e+11,3.457650e+11
2,uid___A002_X1015532_X1926f.small_04,[OBSERVE_TARGET#ON_SOURCE],"(864, 4, 4, 2)","[XX, YY]","[3, 5]",X780709176#ALMA_RB_07#BB_1#SW-01#FULL_RES_0,[Jupiter_1],[Jupiter_1],[SPW1_-_CO_v=0_3-2(ID=3768104)],Ephemeris,3.457649e+11,3.457650e+11
1,uid___A002_X1015532_X1926f.small_05,"[CALIBRATE_ATMOSPHERE#OFF_SOURCE, CALIBRATE_WV...","(30, 4, 4, 2)","[XX, YY]","[2, 4]",X780709176#ALMA_RB_07#BB_2#SW-01#FULL_RES_1,[Jupiter_1],[Jupiter_1],[SPW2(ID=3768104)],Ephemeris,3.450297e+11,3.450765e+11
10,uid___A002_X1015532_X1926f.small_06,"[CALIBRATE_ATMOSPHERE#AMBIENT, CALIBRATE_WVR#A...","(30, 4, 4, 2)","[XX, YY]","[2, 4]",X780709176#ALMA_RB_07#BB_2#SW-01#FULL_RES_1,[Jupiter_1],[Jupiter_1],[SPW2(ID=3768104)],Ephemeris,3.450297e+11,3.450765e+11
15,uid___A002_X1015532_X1926f.small_07,"[CALIBRATE_ATMOSPHERE#HOT, CALIBRATE_WVR#HOT]","(30, 4, 4, 2)","[XX, YY]","[2, 4]",X780709176#ALMA_RB_07#BB_2#SW-01#FULL_RES_1,[Jupiter_1],[Jupiter_1],[SPW2(ID=3768104)],Ephemeris,3.450297e+11,3.450765e+11
12,uid___A002_X1015532_X1926f.small_08,[OBSERVE_TARGET#OFF_SOURCE],"(510, 4, 4, 2)","[XX, YY]","[3, 5]",X780709176#ALMA_RB_07#BB_2#SW-01#FULL_RES_1,[Jupiter_1],[Jupiter_1],[SPW2(ID=3768104)],Ephemeris,3.450297e+11,3.450765e+11
13,uid___A002_X1015532_X1926f.small_09,[OBSERVE_TARGET#ON_SOURCE],"(864, 4, 4, 2)","[XX, YY]","[3, 5]",X780709176#ALMA_RB_07#BB_2#SW-01#FULL_RES_1,[Jupiter_1],[Jupiter_1],[SPW2(ID=3768104)],Ephemeris,3.450297e+11,3.450765e+11


## Inspect antenna_xds

In [11]:
ant_xds = ps['uid___A002_X1015532_X1926f.small_09'].attrs['antenna_xds'].load()
ant_xds

<xarray.Dataset> Size: 988B
Dimensions:                (antenna_name: 4, cartesian_pos_label: 3,
                            receptor_label: 2, sky_dir_label: 2)
Coordinates:
  * antenna_name           (antenna_name) <U9 144B 'PM01_T703' ... 'PM04_T704'
  * cartesian_pos_label    (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    mount                  (antenna_name) <U6 96B 'ALT-AZ' 'ALT-AZ' ... 'ALT-AZ'
    polarization_type      (antenna_name, receptor_label) <U1 32B 'X' ... 'Y'
  * receptor_label         (receptor_label) <U25 200B 'pol_0' 'pol_1'
  * sky_dir_label          (sky_dir_label) <U3 24B 'ra' 'dec'
    station                (antenna_name) <U4 64B 'T703' 'T701' 'T702' 'T704'
Data variables:
    ANTENNA_DISH_DIAMETER  (antenna_name) float64 32B 12.0 12.0 12.0 12.0
    ANTENNA_FEED_OFFSET    (antenna_name, cartesian_pos_label) float64 96B 0....
    ANTENNA_POSITION       (antenna_name, cartesian_pos_label) float64 96B 2....
    BEAM_OFFSET            (antenna_name, receptor_label, sky_dir_label) float64 128B ...
    RECEPTOR_ANGLE         (antenna_name, receptor_label) float64 64B -0.9346...
Attributes:
    overall_telescope_name:  ALMA
    relocatable_antennas:    True